In [12]:
#Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import os
import zipfile
import os
import re


In [ ]:
#Unzip the zip files
zip_path = "court_case_texts.zip"
extract_path = "court_case_texts"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
#Check number of text files
text_files = []

for root, dirs, files in os.walk("court_case_texts"):
    for file in files:
        if file.endswith(".txt"):
            text_files.append(os.path.join(root, file))

length = len(text_files)
print('Length of text files:', length)


Length of text files: 510


In [ ]:
#Create the df for meta data associated
df = pd.read_csv('canlii_final_report_20.csv')
df

,Judge,Heard_Date,Released_Date,Case_Title,URL
0,C.A. Brannagan,27 November 2025,2025-12-31,R. v. M.T.,https://www.canlii.org/en/on/oncj/doc/2025/202...
1,Fergus ODonnell,Unknown,2025-12-30,R. v. J.G.,https://www.canlii.org/en/on/oncj/doc/2025/202...
2,S. Robichaud,"October 27 to October 31, November 12, 17-25, ...",2025-12-29,R. v. Laguerre,https://www.canlii.org/en/on/oncj/doc/2025/202...
3,S. Robichaud,"December 29, 2025",2025-12-29,R. v. Khosa,https://www.canlii.org/en/on/oncj/doc/2025/202...
4,S. G. Pratt,"4 November, 17 December, 2025",2025-12-24,R. v. Lachance,https://www.canlii.org/en/on/oncj/doc/2025/202...
...,...,...,...,...,...
505,Brock Jones,"June 13, 2023, and December 10, 2024",2025-01-06,R. v. Aden,https://www.canlii.org/en/on/oncj/doc/2025/202...
506,Brock Jones,"October 15-17, 28, and December 19, 2024",2025-01-06,R. v. A.B.,https://www.canlii.org/en/on/oncj/doc/2025/202...
507,H. Pringle,"December 6, 2024 [1]",2025-01-03,R. v. Williams,https://www.canlii.org/en/on/oncj/doc/2025/202...
508,Unknown,Unknown,2025-01-02,R. v. Wu,https://www.canlii.org/en/on/oncj/doc/2025/202...


In [ ]:
#Cleaning function 
def clean_court_text_keep_paragraphs(text):
    # 1. Remove everything above "ONTARIO COURT OF JUSTICE"
    if "ONTARIO COURT OF JUSTICE" in text:
        text = "ONTARIO COURT OF JUSTICE\n" + text.split("ONTARIO COURT OF JUSTICE", 1)[1]

    # 2. Remove bracketed numbers like [16], [1], [2] etc.
    text = re.sub(r'\[\s*\d+\s*\]', '', text)
    
    # 3. Replace multiple line breaks with a single line break to keep paragraphs
    text = re.sub(r'\n{2,}', '\n', text)  # keep single line break between paragraphs
    
    # 4. Replace multiple spaces with a single space (within paragraphs)
    text = re.sub(r'[ ]{2,}', ' ', text)
    
    # 5. Lowercase for NLP (optional)
    text = text.lower()
    
    # 6. Strip leading/trailing spaces on each line
    text = '\n'.join([line.strip() for line in text.split('\n') if line.strip() != ''])
    
    return text

# Create folder for cleaned files
clean_folder = "court_case_texts_cleaned"
os.makedirs(clean_folder, exist_ok=True)

# Clean and save each file
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                raw_text = f.read()
            
            cleaned_text = clean_court_text_keep_paragraphs(raw_text)
            
            # Save cleaned text
            cleaned_path = os.path.join(clean_folder, file)
            with open(cleaned_path, "w", encoding="utf-8") as f:
                f.write(cleaned_text)

print(f"All cleaned files saved in: {clean_folder}")

All cleaned files saved in: court_case_texts_cleaned
